In [1]:
import smac
import logging
import numpy as np
from ConfigSpace.hyperparameters import UniformFloatHyperparameter
# Import ConfigSpace and different types of parameters
from smac.configspace import ConfigurationSpace
from smac.facade.smac_bo_facade import SMAC4BO
from smac.facade.smac_hpo_facade import SMAC4HPO
# Import SMAC-utilities
from smac.scenario.scenario import Scenario
import pandas as pd
import time
import cometspy as c
import os
import template_simulation_flavonoids as t
import shutil
import cobra
from cobra import Reaction,Metabolite
import requests
from os.path import exists

In [2]:
dirwork=os.getcwd()

# Model modifications

## E. coli with butanol production

In [ ]:
# Uso del modelo iML1515 , ecoli K12


if not exists("PROMICON/iML1515.xml"):
    url = 'http://bigg.ucsd.edu/static/models/iML1515.xml'
    r = requests.get(url, allow_redirects=True)
    open('iML1515.xml', 'wb').write(r.content)
iML1515=cobra.io.read_sbml_model("PROMICON/iML1515.xml")





In [ ]:
# Reacciones necesarios para la generación de butanol
from cobra import Reaction,Metabolite
def model_modifications_pathway_pyruvate_fermentation(model):
    modified_model=model.copy()
    PYRDC=Reaction("PYRDC")
    PYRDC.name="pyruvate decarboxylase"
    PYRDC.subsystem="butanol biosynthesis (engineered)"
    btal_c=Metabolite("btal_c",name="Butanal",compartment='c',formula='C4H8O',charge=0)
    PYRDC.add_metabolites({modified_model.metabolites.get_by_id("3mob_c"):-1,
                        modified_model.metabolites.get_by_id("h_c"):-1,
                        modified_model.metabolites.get_by_id("co2_c"):1,
                        btal_c:1}) 
    
    PYRDC=Reaction("")
    PYRDC.name="alcohol deydrogenase"
    PYRDC.subsystem="butanol biosynthesis (engineered)"
    PYRDC.add_metabolites({modified_model.metabolites.get_by_id("3hbcoa_c"):-1,
                        modified_model.metabolites.get_by_id("h_c"):-1,
                        modified_model.metabolites.get_by_id("co2_c"):1,
                        btal_c:1}) 
    
    butanal_butanol_modifications(modified_model)
    
def model_modifications_pathway_clostridium(model):
    modified_model=model.copy()
    
    # HBCH reaction
    ctncoa_c=Metabolite("ctncoa_c",name="Crotonyl-CoA",compartment='c',formula='C25H40N7O17P3S',charge=0)
    HBCH=Reaction("HBCH",name="Crotonyl-Coa creation")
    HBCH.lower_bound=-1000
    HBCH.upper_bound=1000
    HBCH.add_metabolites({modified_model.metabolites.get_by_id("3hbcoa_c"):-1,
                        modified_model.metabolites.get_by_id("h_c"):-1,
                        modified_model.metabolites.get_by_id("h2o_c"):1,
                        ctncoa_c:1})
    modified_model.add_reactions([HBCH])
    
    # BCD reaction
    BCD=Reaction("BCD",name="butyryl-CoA dehydrogenase")
    bcoa_c=Metabolite("bcoa_c",name="Butyryl CoA",compartment='c',formula='C25H42N7O17P3S',charge=0)

    BCD.add_metabolites({modified_model.metabolites.get_by_id("ctncoa_c"):-1,
                        modified_model.metabolites.get_by_id("h_c"):-1,
                        modified_model.metabolites.get_by_id("nadh_c"):-1,
                        bcoa_c:1,
                        modified_model.metabolites.get_by_id("nad_c"):1,
                        modified_model.metabolites.get_by_id("coa_c"):1})
    modified_model.add_reactions([BCD])
    
    # BAD reaction
    btal_c=Metabolite("btal_c",name="Butanal",compartment='c',formula='C4H8O',charge=0)
    BAD=Reaction("BAD",name="Butyryl aldehyde dehydrogenase")
    BAD.lower_bound=-1000
    BAD.upper_bound=1000
    BAD.add_metabolites({modified_model.metabolites.get_by_id("bcoa_c"):-1,
                        modified_model.metabolites.get_by_id("nadh_c"):-1,
                        modified_model.metabolites.get_by_id("h_c"):-1, 
                        btal_c:1,
                        modified_model.metabolites.get_by_id("nad_c"):1})
    modified_model.add_reactions([BAD])
    butanal_butanol_modifications(modified_model)
    return(modified_model)
    
def butanal_butanol_modifications(modified_model):
  
    # NADBH reaction
    btoh_c=Metabolite("btoh_c",name="Butanol",compartment='c',formula='C4H10O',charge=0)
    btal_c=Metabolite("btal_c",name="Butanal",compartment='c',formula='C4H8O',charge=0)
    
    NADBH=Reaction("NADBH",name="NADPH-dependent butanol dehydrogenase")
    NADBH.lower_bound=-1000
    NADBH.upper_bound=1000
    NADBH.add_metabolites({btal_c:-1,
                        modified_model.metabolites.get_by_id("nadh_c"):-1,
                        modified_model.metabolites.get_by_id("h_c"):-1,   
                        btoh_c:1,
                        modified_model.metabolites.get_by_id("nad_c"):1
                        })
    modified_model.add_reactions([NADBH])
    
    # BTOHt2rpp reaction
    btoh_p=Metabolite("btoh_p",name="Butanol",compartment='p',formula='C4H10O',charge=0)
    BTOHt2rpp=Reaction("BTOHt2rpp",name="Butanol transport via proton symport (periplasm)")
    BTOHt2rpp.lower_bound=-1000
    BTOHt2rpp.upper_bound=1000
    BTOHt2rpp.add_metabolites({modified_model.metabolites.get_by_id("h_c"):-1,
                        modified_model.metabolites.get_by_id("btoh_c"):-1,
                        btoh_p:1,
                        modified_model.metabolites.get_by_id("h_p"):1
                        })
    modified_model.add_reactions([BTOHt2rpp])
    
    # BTOHtex reaction
    btoh_e=Metabolite("btoh_e",name="Butanol",compartment='e',formula='C4H10O',charge=0)

    BTOHtex=Reaction("BTOHtex",name="Butanol transport via diffusion (extracellular to periplasm)")
    BTOHtex.lower_bound=-1000
    BTOHtex.upper_bound=1000
    BTOHtex.add_metabolites({modified_model.metabolites.get_by_id("btoh_p"):-1,
                        btoh_e:1
                        })
    modified_model.add_reactions([BTOHtex])
    
    # EX_btoh_e reaction
    EX_btoh_e=Reaction("EX_btoh_e",name="Butanol exchange")
    EX_btoh_e.lower_bound=0
    EX_btoh_e.upper_bound=1000
    EX_btoh_e.add_metabolites({modified_model.metabolites.get_by_id("btoh_e"):-1})
    modified_model.add_reactions([EX_btoh_e])


In [ ]:
iML1515_butanol=model_modifications_pathway_clostridium(iML1515)

In [ ]:
iML1515_butanol.reactions.EX_glc__D_e.lower_bound=0
iML1515_butanol.reactions.EX_ac_e.lower_bound=-10
iML1515_butanol.summary()

In [ ]:
from cobra.flux_analysis import flux_variability_analysis
cobra.flux_analysis.flux_variability_analysis(
    iML1515_butanol, "EX_btoh_e", fraction_of_optimum=0.9)


In [ ]:
cobra.io.write_sbml_model(iML1515_butanol,"PROMICON/iML1515_butanol.xml")

## Synechocystis acetate productor

In [ ]:
# Synechocystis acetate productor
import importExcelModel
if not exists("PROMICON/iJN678_new_annotation.mat"):
    print("No esta el modelo")
else:
    iJN678=importExcelModel.import_excel_model("PROMICON/iJN678.xls","iJN678")


In [ ]:
iJN678.summary()


In [ ]:
cobra.io.write_sbml_model(iJN678,"PROMICON/iJN678_acetate.xml")

### Mutations for acetate production based on https://biotechnologyforbiofuels.biomedcentral.com/articles/10.1186/s13068-018-1037-8

In [ ]:
iJN678.reactions.ACS.knock_out()
iJN678.reactions.ACKr.knock_out()
iJN678.reactions.ACtex.bounds=(-1000,1000)
iJN678.reactions.EX_ac_e.bounds=(0,1000)
iJN678.summary()

In [ ]:
cobra.io.write_sbml_model(iJN678,"PROMICON/iJN678_acetate_mutant.xml")

## Synechoccocus sucrose productor

In [ ]:
iJN785=cobra.io.read_sbml_model("PROMICON/iJB785.xml")
iJN785.reactions.EX_leu__L_e.lower_bound=0
iJN785.summary()

In [ ]:
def add_sucrose_transporters(model):
    sucr_p=Metabolite("sucr_p",
                     name="Sucrose",
                     compartment="p",
                     formula="C12H22O11",
                     charge=0)
    sucr_e=Metabolite("sucr_e",
                     name="Sucrose",
                     compartment="e",
                     formula="C12H22O11",
                     charge=0)
    SUCRtpp=Reaction("SUCRtpp",name="Sucrose transport via proton symport (periplasm)")
    SUCRtpp.lower_bound=0
    SUCRtpp.upper_bound=1000
    SUCRtpp.add_metabolites({model.metabolites.get_by_id("h_c"):-1,
                        model.metabolites.get_by_id("sucr_c"):-1,
                        sucr_p:1,
                        model.metabolites.get_by_id("h_p"):1
                        })
    model.add_reactions([SUCRtpp])
    
    # SUCRtex reaction

    SUCRtex=Reaction("SUCRtex",name="Sucrose transport via diffusion (extracellular to periplasm)")
    SUCRtex.lower_bound=0
    SUCRtex.upper_bound=1000
    SUCRtex.add_metabolites({model.metabolites.get_by_id("sucr_p"):-1,
                        sucr_e:1
                        })
    model.add_reactions([SUCRtex])
    
    # EX_sucr_e reaction
    EX_sucr_e=Reaction("EX_sucr_e",name="Butanol exchange")
    EX_sucr_e.lower_bound=0
    EX_sucr_e.upper_bound=1000
    EX_sucr_e.add_metabolites({model.metabolites.get_by_id("sucr_e"):-1})
    model.add_reactions([EX_sucr_e])
    

In [ ]:
add_sucrose_transporters(iJN785)
iJN785.summary()

In [ ]:
cobra.io.write_sbml_model(iJN785,"PROMICON/iJN785_sucrose.xml")

### Modifications based on SPP overexpression and SUCR deletion

In [ ]:
iJN785.reactions.SPP.lower_bound=0.25
iJN785.reactions.SUCR.knock_out()
iJN785.summary()

# R. palustris

iAN1128 downloaded from: Navid, Ali, et al. "System-level analysis of metabolic trade-offs during anaerobic photoheterotrophic growth in Rhodopseudomonas palustris." BMC bioinformatics 20.1 (2019): 1-16.
https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-2844-z

## Modificación del modelo original, NO EJECUTAR

In [ ]:
iAN1128=cobra.io.read_sbml_model("PROMICON/iAN1128.xml")

In [ ]:
def kegg_to_bigg(model):
    kegg2bigg={
        'C00811XT_e':'T4hcinnm_e',
        'C00156XT_e':'4hbz_e',
        'XC54XT_e':'Biomass_e',
        'C00175XT_e':'cobalt2_e',
        'C00033XT_e':'ac_e',
        'C00194XT_e':'adocbl_e',
        'C00026XT_e':'akg_e',
        'C00993XT_e':'alaala_e',
        'C00062XT_e':'arg__L_e',
        'C00152XT_e':'asn__L_e',
        'C00049XT_e':'asp__L_e',
        'C00180XT_e':'bz_e',
        'C00120XT_e':'btn_e',
        'C00076XT_e':'ca2_e',
        'C00853XT_e':'cbl1_e',
        'C01413XT_e':'cd2_e',
        'C00114XT_e':'chol_e',
        'C00698XT_e':'cl_e',
        'C00011XT_e':'co2_e',
        'C00070XT_e':'cu2_e',
        'C00097XT_e':'cys__L_e',
        'C05198XT_e':'dad_5_e',
        'C14818XT_e':'fe2_e',
        'C14819XT_e':'fe3_e',
        'C00095XT_e':'fru_e',
        'C00122XT_e':'fum_e',
        'C00064XT_e':'gln__L_e',
        'C00025XT_e':'glu__L_e',
        'C00719XT_e':'glyb_e',
        'C00093XT_e':'glyc3p_e',
        'C00116XT_e':'glyc_e',
        'C00127XT_e':'gthox_e',
        'C00051XT_e':'gthrd_e',
        'C00001XT_e':'h2o_e',
        'C00282XT_e':'h2_e',
        'C00135XT_e':'his__L_e',
        'C00205XT_e':'photon_e',
        'C00080XT_e':'h_e',
        'C00407XT_e':'ile__L_e',
        'C00238XT_e':'k_e',
        'C00186XT_e':'lac__L_e',
        'C00123XT_e':'leu__L_e',
        'C00047XT_e':'lys__L_e',
        'C00149XT_e':'mal__L_e',
        'C00208XT_e':'malt_e',
        'C00159XT_e':'man_e',
        'C00073XT_e':'met__L_e',
        'C00305XT_e':'mg2_e',
        'C00034XT_e':'mn2_e',
        'C06232XT_e':'mobd_e',
        'C00697XT_e':'n2_e',
        'C01330XT_e':'na1_e',
        'C00014XT_e':'nh3_e',
        'C00244XT_e':'no3_e',
        'C00007XT_e':'o2_e',
        'C06696XT_e':'pb_e',
        'C00032XT_e':'pheme_e',
        'C00146XT_e':'phenol_e',
        'C02656XT_e':'pime_e',
        'C00009XT_e':'pi_e',
        'C00864XT_e':'pnto__R_e',
        'C00744XT_e':'pro__L_e',
        'C00134XT_e':'ptrc_e',
        'C00022XT_e':'pyr_e',
        'C00121XT_e':'rib__D_e',
        'C00506XT_e':'Lcyst_e',
        'C00059XT_e':'so4_e',
        'C00315XT_e':'spmd_e',
        'C00042XT_e':'succ_e',
        'C00089XT_e':'sucr_e',
        'C14179XT_e':'sula_e',
        'C00245XT_e':'taur_e',
        'C00378XT_e':'thm_e',
        'C01083XT_e':'tre_e',
        'C00320XT_e':'tsul_e',
        'C00086XT_e':'urea_e',
        'C00183XT_e':'val__L_e',
        'C06672XT_e':'vanlt_e',
        'C00038XT_e':'zn2_e',
        'C00811XT_c':'T4hcinnm_c',
        'C00156_c':'4hbz_c',
        'XC54_c':'Biomass_c',
        'C00175_c':'cobalt2_c',
        'C00033_c':'ac_c',
        'C00194_c':'adocbl_c',
        'C00026_c':'akg_c',
        'C00993_c':'alaala_c',
        'C00062_c':'arg__L_c',
        'C00152_c':'asn__L_c',
        'C00049_c':'asp__L_c',
        'C00180_c':'bz_c',
        'C00120_c':'btn_c',
        'C00076_c':'ca2_c',
        'C00853_c':'cbl1_c',
        'C01413_c':'cd2_c',
        'C00114_c':'chol_c',
        'C00698_c':'cl_c',
        'C00011_c':'co2_c',
        'C00070_c':'cu2_c',
        'C00097_c':'cys__L_c',
        'C05198_c':'dad_5_c',
        'C14818_c':'fe2_c',
        'C14819_c':'fe3_c',
        'C00095_c':'fru_c',
        'C00122_c':'fum_c',
        'C00064_c':'gln__L_c',
        'C00025_c':'glu__L_c',
        'C00719_c':'glyb_c',
        'C00093_c':'glyc3p_c',
        'C00116_c':'glyc_c',
        'C00127_c':'gthox_c',
        'C00051_c':'gthrd_c',
        'C00001_c':'h2o_c',
        'C00282_c':'h2_c',
        'C00135_c':'his__L_c',
        'C00205_c':'photon_c',
        'C00080_c':'h_c',
        'C00407_c':'ile__L_c',
        'C00238_c':'k_c',
        'C00186_c':'lac__L_c',
        'C00123_c':'leu__L_c',
        'C00047_c':'lys__L_c',
        'C00149_c':'mal__L_c',
        'C00208_c':'malt_c',
        'C00159_c':'man_c',
        'C00073_c':'met__L_c',
        'C00305_c':'mg2_c',
        'C00034_c':'mn2_c',
        'C06232_c':'mobd_c',
        'C00697_c':'n2_c',
        'C01330_c':'na1_c',
        'C00014_c':'nh3_c',
        'C00244_c':'no3_c',
        'C00007_c':'o2_c',
        'C06696_c':'pb_c',
        'C00032_c':'pheme_c',
        'C00146_c':'phenol_c',
        'C02656_c':'pime_c',
        'C00009_c':'pi_c',
        'C00864_c':'pnto__R_c',
        'C00744_c':'pro__L_c',
        'C00134_c':'ptrc_c',
        'C00022_c':'pyr_c',
        'C00121_c':'rib__D_c',
        'C00506_c':'Lcyst_c',
        'C00059_c':'so4_c',
        'C00315_c':'spmd_c',
        'C00042_c':'succ_c',
        'C00089_c':'sucr_c',
        'C14179_c':'sula_c',
        'C00245_c':'taur_c',
        'C00378_c':'thm_c',
        'C01083_c':'tre_c',
        'C00320_c':'tsul_c',
        'C00086_c':'urea_c',
        'C00183_c':'val__L_c',
        'C06672_c':'vanlt_c',
        'C00038_c':'zn2_c',
        'C00811PX_p':'T4hcinnm_p',
        'C00156PX_p':'4hbz_p',
        'XC54PX_p':'Biomass_p',
        'C00175PX_p':'cobalt2_p',
        'C00033PX_p':'ac_p',
        'C00194PX_p':'adocbl_p',
        'C00026PX_p':'akg_p',
        'C00993PX_p':'alaala_p',
        'C00062PX_p':'arg__L_p',
        'C00152PX_p':'asn__L_p',
        'C00049PX_p':'asp__L_p',
        'C00180PX_p':'bz_p',
        'C00120PX_p':'btn_p',
        'C00076PX_p':'ca2_p',
        'C00853PX_p':'cbl1_p',
        'C01413PX_p':'cd2_p',
        'C00114PX_p':'chol_p',
        'C00698PX_p':'cl_p',
        'C00011PX_p':'co2_p',
        'C00070PX_p':'cu2_p',
        'C00097PX_p':'cys__L_p',
        'C05198PX_p':'dad_5_p',
        'C14818PX_p':'fe2_p',
        'C14819PX_p':'fe3_p',
        'C00095PX_p':'fru_p',
        'C00122PX_p':'fum_p',
        'C00064PX_p':'gln__L_p',
        'C00025PX_p':'glu__L_p',
        'C00719PX_p':'glyb_p',
        'C00093PX_p':'glyc3p_p',
        'C00116PX_p':'glyc_p',
        'C00127PX_p':'gthox_p',
        'C00051PX_p':'gthrd_p',
        'C00001PX_p':'h2o_p',
        'C00282PX_p':'h2_p',
        'C00135PX_p':'his__L_p',
        'C00205PX_p':'photon_p',
        'C00080PX_p':'h_p',
        'C00407PX_p':'ile__L_p',
        'C00238PX_p':'k_p',
        'C00186PX_p':'lac__L_p',
        'C00123PX_p':'leu__L_p',
        'C00047PX_p':'lys__L_p',
        'C00149PX_p':'mal__L_p',
        'C00208PX_p':'malt_p',
        'C00159PX_p':'man_p',
        'C00073PX_p':'met__L_p',
        'C00305PX_p':'mg2_p',
        'C00034PX_p':'mn2_p',
        'C06232PX_p':'mobd_p',
        'C00697PX_p':'n2_p',
        'C01330PX_p':'na1_p',
        'C00014PX_p':'nh3_p',
        'C00244PX_p':'no3_p',
        'C00007PX_p':'o2_p',
        'C06696PX_p':'pb_p',
        'C00032PX_p':'pheme_p',
        'C00146PX_p':'phenol_p',
        'C02656PX_p':'pime_p',
        'C00009PX_p':'pi_p',
        'C00864PX_p':'pnto__R_p',
        'C00744PX_p':'pro__L_p',
        'C00134PX_p':'ptrc_p',
        'C00022PX_p':'pyr_p',
        'C00121PX_p':'rib__D_p',
        'C00506PX_p':'Lcyst_p',
        'C00059PX_p':'so4_p',
        'C00315PX_p':'spmd_p',
        'C00042PX_p':'succ_p',
        'C00089PX_p':'sucr_p',
        'C14179PX_p':'sula_p',
        'C00245PX_p':'taur_p',
        'C00378PX_p':'thm_p',
        'C01083PX_p':'tre_p',
        'C00320PX_p':'tsul_p',
        'C00086PX_p':'urea_p',
        'C00183PX_p':'val__L_p',
        'C06672PX_p':'vanlt_p',
        'C00038PX_p':'zn2_p'
 
 
    }
    for metabolite in model.metabolites:
        if metabolite.id in kegg2bigg:
            metabolite.id=kegg2bigg[metabolite.id]
    importExcelModel.cobrapy_to_excel(model,"PROMICON/iAN1128_bigg_tmp.xlsx")
    

In [ ]:
iAN1128_old=importExcelModel.import_excel_model("PROMICON/iAN1128_fixed.xls","iAN1128")

In [ ]:
iAN1128_old.summary()

In [ ]:
kegg_to_bigg(iAN1128_old)


### El modelo necesita cambios manuales de nombres de reacciones y se ha añadido HEX1

In [ ]:
iAN1128=importExcelModel.import_excel_model("PROMICON/iAN1128_bigg.xlsx","iAN1128")
iAN1128.summary()

In [ ]:
# Importe de N2 y limitacion de NH3 para usarlo para producir H2
iAN1128.reactions.EX_nh3_e.lower_bound=0
iAN1128.summary()

### hydrogen:ferredoxin oxidoreductase knock out para no producir protones a partir de h2

In [ ]:

iAN1128.reactions.R00019.knock_out()
iAN1128.summary()

In [ ]:
cobra.io.write_sbml_model(iAN1128,"PROMICON/iAN1128_hydrogen.xml")

In [ ]:
def add_sucrose_metabolism_palustris(model):
    INVERTASE=Reaction('INVERTASE')
    INVERTASE.name='Sucrose transformation to fructose and glucose'
    INVERTASE.lower_bound=0
    INVERTASE.upper_bound=1000
    INVERTASE.add_metabolites({model.metabolites.get_by_id("h2o_p"):-1.0,
                               model.metabolites.get_by_id("sucr_p"):-1.0,
                               model.metabolites.get_by_id("C00031_c"):1.0,
                               model.metabolites.get_by_id("fru_p"):1.0
                              })

    model.add_reaction(INVERTASE)
    del(INVERTASE)
    glc__D_e=Metabolite("glc__D_e",
                     name="D-glucose",
                     compartment="e",
                     formula="X",
                     charge=0)
    
    GLCtex=Reaction("GLCtex",name="Glucose transporter")
    GLCtex.lower_bound=0
    GLCtex.upper_bound=1000
    GLCtex.add_metabolites({model.metabolites.get_by_id("C00031_c"):-1,
                                glc__D_e:1.0})
    model.add_reactions([GLCtex])
    EX_glc__D_e=Reaction("EX_glc__D_e",name="Glucose exchange")
    EX_glc__D_e.lower_bound=0
    EX_glc__D_e.upper_bound=1000
    EX_glc__D_e.add_metabolites({model.metabolites.get_by_id("C00031_c"):-1})
    model.add_reactions([EX_glc__D_e])
    

model=cobra.io.read_sbml_model("PROMICON/iAN1128_hydrogen.xml")
add_sucrose_metabolism_palustris(model)
model.reactions.EX_sucr_e.lower_bound=-1000
model.summary()

In [ ]:
cobra.io.write_sbml_model(model,"PROMICON/iAN1128_sucrose.xml")

# P.putida basado en el modelo de Manoli

In [ ]:
# Import P. putida 
pputida=cobra.io.read_sbml_model("PROMICON/modelManoli.xml")
pputida.summary()

In [ ]:
def add_sucrose_metabolism(model):

    # Reaccion de transporte de sacarosa
    sucr_e = Metabolite('sucr_e', compartment='e',formula='C12H22O11', name='Sucrose C12H22O11')
    sucr_p = Metabolite('sucr_p', compartment='p',formula='C12H22O11', name='Sucrose C12H22O11')
    sucr_c = Metabolite('sucr_c', compartment='c',formula='C12H22O11', name='Sucrose C12H22O11')
    EX_sucr_e=Reaction('EX_sucr_e')
    EX_sucr_e.name='Sucrose exchange'
    EX_sucr_e.lower_bound=-10
    EX_sucr_e.upper_bound=0
    EX_sucr_e.add_metabolites({sucr_e:-1.0})
    
    model.add_reaction(EX_sucr_e)
    SUCRtex=Reaction('SUCRtex')
    SUCRtex.name='Sucrose transport (extracelular to periplasm)'
    SUCRtex.lower_bound=0
    SUCRtex.upper_bound=1000
    SUCRtex.add_metabolites({model.metabolites.get_by_id("sucr_e"):-1.0,
                               sucr_p:1.0})
    model.add_reaction(SUCRtex)
    del(SUCRtex)
    
    SUCRtpp=Reaction('SUCRtpp')
    SUCRtpp.name='Sucrose transport (periplasm to cytoplasm)'
    SUCRtpp.lower_bound=0
    SUCRtpp.upper_bound=1000
    SUCRtpp.add_metabolites({model.metabolites.get_by_id("sucr_p"):-1.0,
                               model.metabolites.get_by_id("h_p"):-1.0,
                               sucr_c:1.0,
                               model.metabolites.get_by_id("h_c"):1.0})
    model.add_reaction(SUCRtpp)
    del(SUCRtpp)
    
    INVERTASE=Reaction('INVERTASE')
    INVERTASE.name='Sucrose transformation to fructose'
    INVERTASE.lower_bound=0
    INVERTASE.upper_bound=1000
    INVERTASE.add_metabolites({model.metabolites.get_by_id("h2o_p"):-1.0,
                               model.metabolites.get_by_id("sucr_p"):-1.0,
                               model.metabolites.get_by_id("glc__D_p"):1.0,
                               model.metabolites.get_by_id("fru_p"):1.0
                              })

    model.add_reaction(INVERTASE)
    del(INVERTASE)

def add_PHA_transporter(model):
    # Required transport reactions for COMETS
    C80aPHA_e = Metabolite('C80aPHA_e', compartment='e',formula='C8H15O3R', name='PHA')
    C80aPHAtex=Reaction('C80aPHAtex')
    C80aPHAtex.name='C80aPHA transport (extracelular to cytoplasm)'
    C80aPHAtex.lower_bound=0
    C80aPHAtex.upper_bound=1000
    C80aPHAtex.add_metabolites({model.metabolites.get_by_id("C80aPHA_c"):-1.0,
                               C80aPHA_e:1.0})
    model.add_reaction(C80aPHAtex)
    EX_C80aPHA_e=Reaction('EX_C80aPHA_e')
    EX_C80aPHA_e.name='C80aPHA EXCHANGE'
    EX_C80aPHA_e.lower_bound=0
    EX_C80aPHA_e.upper_bound=1000
    EX_C80aPHA_e.add_metabolites({model.metabolites.get_by_id("C80aPHA_e"):-1.0})
    model.add_reaction(EX_C80aPHA_e)
    # Disable demand reaction
    model.reactions.DM_C80aPHA.knock_out()
    
    

In [ ]:
add_sucrose_metabolism(pputida)
add_PHA_transporter(pputida)

In [ ]:
# Quitamos pet como fuente de carbono y le metemos sacarosa
pputida.reactions.EX_pet_e.lower_bound=0

In [ ]:
pputida.summary()

In [ ]:
cobra.io.write_sbml_model(pputida,"PROMICON/manoli_sucrose.xml")

# Con los modelos ejecutamos COMETS

### Funciones auxiliares para ejecutar comets

In [5]:
import cobra
import pandas as pd
import os.path
import os
import copy
import shutil, errno
import statistics
import cometspy as c
from cobra import Reaction
from cobra import Metabolite
import matplotlib.pyplot as plt
import numpy as np

################################################################
### FUNCTION SynBio4FlavFLYCOP_oneConf ############################
def change_infite_value_comets(cometsfile):
    #read input file
    fin = open(cometsfile, "rt")
    #read file contents to string
    data = fin.read()
    #replace all occurrences of the required string
    data = data.replace('inf', '1000')
    #close the input file
    fin.close()
    #open the input file in write mode
    fin = open(cometsfile, "wt")
    #overrite the input file with the resulting data
    fin.write(data)
    #close the file
    fin.close()

def ensure_sinks_are_not_exchanges(model):
    rxn_names = ['sink_PHAg','SK_pqqA_kt_c']
    for rxn in rxn_names:
        model.reactions.loc[model.reactions.REACTION_NAMES == rxn, 'EXCH'] = False
        model.reactions.loc[model.reactions.REACTION_NAMES == rxn, 'EXCH_IND'] = 0
        
def initialize_layout(models):
    #The layout is a file with the stablished format
    layout=c.layout()
    for model in models:
        ensure_sinks_are_not_exchanges(model)
        layout.add_model(model)
    layout.add_typical_trace_metabolites()
    layout.update_models()
    return(layout)

def intersection_exchange_metabolites(models):
    # Calculo del medio de varias especies
    all_exchanged_mets = []
    for m in models:
        all_exchanged_mets.extend(m.get_exchange_metabolites())
    all_exchanged_mets = sorted(list(set(list(all_exchanged_mets))))
    return(all_exchanged_mets)


    
    
def create_media(models):
    metabolite_list=intersection_exchange_metabolites(models)
    media_dict=dict()
    # By default 0 ammount
    for metabolite in metabolite_list:
        media_dict[metabolite]=10.0
    media_dict["ac_e"]=0.0
    media_dict["o2_e"]=1000.0
    media_dict["nh3_e"]=30.0
    media_dict["ca2_e"]=1000
    media_dict["co2_e"]=100
    media_dict["cobalt2_e"]=1000
    media_dict["cu2_e"]=1000
    media_dict["fe2_e"]=100
    media_dict["fe3_e"]=100
    media_dict["h_e"]=100
    media_dict["h2o_e"]=1000
    media_dict["k_e"]=100
    media_dict["mg2_e"]=100
    media_dict["mn2_e"]=100
    media_dict["mobd_e"]=100
    media_dict["na1_e"]=100
    media_dict["no3_e"]=100
    media_dict["photon_e"]=1000
    media_dict["pi_e"]=100
    media_dict["so4_e"]=100
    media_dict["zn2_e"]=1000
    media_dict["btn_e"]=40
    media_dict["cl_e"]=1000
    media_dict["nh4_e"]=40
    media_dict["ni2_e"]=1000

    # Return pandas dataframe
    return(pd.DataFrame(media_dict.items(),columns=['metabolite','init_amount']))
    

def initialize_params(package, globall):
    """Function to initialize the comets' params class
    it can be initialize by submitting two files, one for the package parameters
    and one for the global ones.
    If you don't submit a file, the params class will be initialize with the values stated below
    which have been tested in this simulation"""
    
    if package and globall is not None:
        params = c.params(global_params = globall, package_params= package)
    elif package is not None:
        params = c.params(package_params=package)
    elif globall is not None:
        params = c.params(global_params=globall)
        
    else:
        params = c.params()
        params.all_params['maxCycles']=1000
        params.all_params['timeStep']=0.1
        params.all_params['spaceWidth']=0.05
        params.all_params['allowCellOverlap']= True
        params.all_params['cellSize'] = 4.3e-13
        params.all_params['deathRate']= 0.00
        params.all_params['dilFactor'] = 1e-2
        params.all_params['dilTime'] = 12
        params.all_params['numRunThreads']= 8
        params.all_params['maxSpaceBiomass']= 100
        params.all_params['defaultVmax']= 20
        params.all_params['defaultKm']= 0.01
        params.all_params['defaultHill']= 1
        params.all_params['showCycleTime']= True
        params.all_params['geneFractionalCost']= 0
        params.all_params['writeTotalBiomassLog']= True
        params.all_params['writeMediaLog']= True
        params.all_params['writeFluxLog']= True
        params.all_params['useLogNameTimeStamp']= False
        params.all_params['flowDiffRate']= 1e-7
        params.all_params['growthDiffRate']=1e-7
        params.all_params['FluxLogRate']= 1
        params.all_params['MediaLogRate']= 1
        params.all_params['numExRxnSubsteps']= 12
        params.all_params['geneFractionalCost'] = 0
        params.all_params['exchangestyle']= 'Standard FBA'
        params.all_params['biomassMotionStyle']= 'Diffusion 2D(Crank-Nicolson)'
        params.all_params['mutRate'] = 1e-9
        params.all_params['addRate'] = 1e-9
        params.all_params['minSpaceBiomass'] = 1e-10

        
    #check if param 'maxSpaceBiomass' has default value
    if (params.all_params['maxSpaceBiomass']== 0.1):
        print('The parameter "maxSpaceBiomass" is set to 0.1.\n' \
              'It may need to change if the mo used growths well.')

    return params

def find_end_cycle(simulation_output):
    # it finish when the strains stop the growth
    end_cycle=0
    counter=0
    for index,row in simulation_output.iterrows():
        new_biomasses=row.iloc[1:]
        if index==0 or index==1:
            old_biomasses=row.iloc[1:]
        else:
            result=new_biomasses.subtract(old_biomasses)
            is_growing=result.apply(lambda x: x>0)
            old_biomasses=new_biomasses
            if is_growing.any()==False:
                counter=counter+1
                if counter==10:
                    break
        end_cycle=index
    return(end_cycle)

def make_df_and_graph(strains, metabolites, comets, max_cycles):
    '''This function creates a figure and saves it to pdf format.
    It also creates the file biomass_vs_met.txt which contais the quantity
    of each strain and metabolite and has the following columns:
    time(h), strain1 ... strainX, met1 ... metX.'''
    file_name='_'.join(metabolites)
    df = comets.media #We get the media composition results'
    df_media=copy.deepcopy(df.loc[df['cycle']<max_cycles])
    df2=comets.total_biomass #We get the biomass results
    df_biomass=copy.deepcopy(df2.loc[df2['cycle']<max_cycles])
    columns=['cycle']
    for i in range(0,len(strains)):
        columns.append(strains[i])
    df_biomass.columns=columns
    
    """For each metabolite a column with all zeros is added to the dataframe and each row that contains a value
     (metabolite concentration)is changed in the dataframe"""
    for d in metabolites:
        columns.append(d)
        met =df_media.loc[df_media['metabolite'] == d]

        temp=np.zeros(max_cycles) #Create an array with all zeros
        df_biomass[d]=temp #We added it to the dataframe
        j=1
        while j < (max_cycles): #For each cycle
            if (met.cycle==j).any(): #If the row exists
                df_biomass.loc[j-1,d] = float(met[met['cycle']==j]['conc_mmol']) #Its dataframe value is changed
            j+=1
    df_biomass.columns=columns
    np.savetxt(r'biomass_vs_'+file_name+'_template.txt', df_biomass.values, fmt='%s',delimiter='\t',header='\t'.join(columns)) #The data is saved
    #---Starting the figure 
    plt.ioff()
    fig, ax = plt.subplots()

    ax.set_xlabel('time (h)')
    ax.set_ylabel('biomass (g/L)')
    c=['k', 'm', 'b', 'g', 'r']
    j=0
    for i in strains:
        ax.plot(df_biomass['cycle']*0.1, df_biomass[i], label=i, color=c[j])
        j+=1
    ax2 = ax.twinx()
    ax2.set_ylabel('metabolite conc (mM)')
    for m in metabolites:
        ax2.plot(df_biomass['cycle']*0.1, df_biomass[m], label=m)

    handles, labels = ax.get_legend_handles_labels()
    handle_list, label_list = [], []

    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)
    handles, labels = ax2.get_legend_handles_labels()
    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)
    plt.legend(handle_list, label_list)
    
    #saving the figure to a pdf
    plt.savefig('biomass_vs_'+file_name+'_template_plot.pdf')
    
    return df_biomass



In [4]:
def set_layout_media_ecoli(layout):
    layout.set_specific_metabolite("o2_e",1000)
    layout.set_specific_metabolite("ac_e",0.0)
    layout.set_specific_metabolite("glc__D_e",0.0)
    layout.set_specific_metabolite("o2_e",100.0)
    layout.set_specific_metabolite("ca2_e",1000)
    layout.set_specific_metabolite("co2_e",100)
    layout.set_specific_metabolite("cobalt2_e",1000)
    layout.set_specific_metabolite("cu2_e",1000)
    layout.set_specific_metabolite("fe2_e",100)
    layout.set_specific_metabolite("fe3_e",100)
    layout.set_specific_metabolite("h_e",100)
    layout.set_specific_metabolite("h2o_e",1000)
    layout.set_specific_metabolite("k_e",100)
    layout.set_specific_metabolite("mg2_e",100)
    layout.set_specific_metabolite("mn2_e",100)
    layout.set_specific_metabolite("mobd_e",100)
    layout.set_specific_metabolite("na1_e",100)
    layout.set_specific_metabolite("no3_e",100)
    layout.set_specific_metabolite("pi_e",100)
    layout.set_specific_metabolite("so4_e",100)
    layout.set_specific_metabolite("zn2_e",1000)
    layout.set_specific_metabolite("btn_e",40)
    layout.set_specific_metabolite("cl_e",1000)
    layout.set_specific_metabolite("nh4_e",40)
    layout.set_specific_metabolite("ni2_e",1000)
    layout.set_specific_metabolite("btn_e",100)
    
def set_layout_media_synechocystis(layout):
    layout.set_specific_metabolite("o2_e",1000)
    layout.set_specific_metabolite("ac_e",0.0)
    layout.set_specific_metabolite("glc__D_e",0.0)
    layout.set_specific_metabolite("o2_e",100.0)
    layout.set_specific_metabolite("ca2_e",1000)
    layout.set_specific_metabolite("co2_e",100)
    layout.set_specific_metabolite("cobalt2_e",1000)
    layout.set_specific_metabolite("cu2_e",1000)
    layout.set_specific_metabolite("fe2_e",100)
    layout.set_specific_metabolite("fe3_e",100)
    layout.set_specific_metabolite("h_e",100)
    layout.set_specific_metabolite("h2o_e",1000)
    layout.set_specific_metabolite("k_e",100)
    layout.set_specific_metabolite("mg2_e",100)
    layout.set_specific_metabolite("mn2_e",100)
    layout.set_specific_metabolite("mobd_e",100)
    layout.set_specific_metabolite("na1_e",100)
    layout.set_specific_metabolite("no3_e",100)
    layout.set_specific_metabolite("pi_e",100)
    layout.set_specific_metabolite("so4_e",100)
    layout.set_specific_metabolite("zn2_e",1000)
    layout.set_specific_metabolite("nh4_e",40)
    layout.set_specific_metabolite("ni2_e",1000)
    layout.set_specific_metabolite("photon_e",1000) 
    

def set_layout_media_synechoccus(layout):
    layout.set_specific_metabolite("o2_e",1000)
    layout.set_specific_metabolite("ca2_e",1000)
    layout.set_specific_metabolite("co2_e",100)
    layout.set_specific_metabolite("cobalt2_e",1000)
    layout.set_specific_metabolite("cu2_e",1000)
    layout.set_specific_metabolite("fe2_e",100)
    layout.set_specific_metabolite("fe3_e",100)
    layout.set_specific_metabolite("h_e",100)
    layout.set_specific_metabolite("h2o_e",1000)
    layout.set_specific_metabolite("k_e",100)
    layout.set_specific_metabolite("mg2_e",100)
    layout.set_specific_metabolite("mn2_e",100)
    layout.set_specific_metabolite("mobd_e",100)
    layout.set_specific_metabolite("na1_e",100)
    layout.set_specific_metabolite("no3_e",100)
    layout.set_specific_metabolite("pi_e",100)
    layout.set_specific_metabolite("so4_e",100)
    layout.set_specific_metabolite("zn2_e",1000)
    layout.set_specific_metabolite("nh4_e",40)
    layout.set_specific_metabolite("ni2_e",1000)
    layout.set_specific_metabolite("photon650_e",1000)
    layout.set_specific_metabolite("photon670_e",1000)
    layout.set_specific_metabolite("photon690_e",1000)
    layout.set_specific_metabolite("hco3_e",1000)
    
def set_layout_media_putida(layout):
    layout.set_specific_metabolite("o2_e",1000)
    layout.set_specific_metabolite("sucr_e",1000)
    layout.set_specific_metabolite("ca2_e",1000)
    layout.set_specific_metabolite("co2_e",100)
    layout.set_specific_metabolite("cobalt2_e",1000)
    layout.set_specific_metabolite("cu2_e",1000)
    layout.set_specific_metabolite("fe2_e",100)
    layout.set_specific_metabolite("fe3_e",100)
    layout.set_specific_metabolite("h_e",100)
    layout.set_specific_metabolite("h2o_e",1000)
    layout.set_specific_metabolite("k_e",100)
    layout.set_specific_metabolite("mg2_e",100)
    layout.set_specific_metabolite("mn2_e",100)
    layout.set_specific_metabolite("mobd_e",100)
    layout.set_specific_metabolite("na1_e",100)
    layout.set_specific_metabolite("no3_e",100)
    layout.set_specific_metabolite("pi_e",100)
    layout.set_specific_metabolite("so4_e",100)
    layout.set_specific_metabolite("zn2_e",1000)
    layout.set_specific_metabolite("nh4_e",40)
    layout.set_specific_metabolite("cl_e",1000)
    layout.set_specific_metabolite("ni2_e",1000)

def set_layout_media_palustris(layout):
    layout.set_specific_metabolite("cl_e",1000)
    #layout.set_specific_metabolite("photon_e",1000)
    layout.set_specific_metabolite("n2_e",1000)
    layout.set_specific_metabolite("ac_e",1000)
    


## Consorcio Synechocystis-E.coli para producción de butanol

In [ ]:
farmer=c.model()
farmer.read_cobra_model('PROMICON/iJN678_acetate_mutant.xml')
producer=c.model()
producer.read_cobra_model('PROMICON/iML1515_butanol.xml')
producer.change_bounds('BTOHtex',0.25,1000)
producer.change_bounds('EX_btoh_e',0,0.25)


In [ ]:
farmer.initial_pop=[0,0,0.2]
producer.initial_pop=[0,0,0.01]
layout=initialize_layout([producer,farmer])
set_layout_media_ecoli(layout)
set_layout_media_synechocystis(layout)
layout.set_specific_metabolite('rdmbzi_c',100)
print("Loading the comets parameters...")
params_package=None
params_global=None
params =initialize_params(params_package, params_global)
print("Creating the comets object...")
comets = c.comets(layout, params)
dirPlot='promicon_output'
comets.run(delete_files=True)
print(comets.run_output)

In [ ]:
metabolites=['co2_e','o2_e','ac_e', 'btoh_e']
df2=make_df_and_graph(["iML1515 butanol","iJN678 acetate"], metabolites, comets, 1000)

In [ ]:
df2

# Consorcio Syneco - Putida

In [ ]:

farmer=c.model()
farmer.read_cobra_model("PROMICON/iJN785_sucrose.xml")
farmer.change_bounds('SPP',0.25,1000)
farmer.change_bounds('SUCR',0,0)

producer=c.model()
producer.read_cobra_model("PROMICON/manoli_sucrose.xml")
producer.change_bounds('C80aPHAtex',2,1000)
producer.change_bounds('EX_C80aPHA_e',2,1000)

farmer.initial_pop=[0,0,0.2]
producer.initial_pop=[0,0,0.01]
layout=initialize_layout([producer,farmer])
set_layout_media_putida(layout)
set_layout_media_synechoccus(layout)

#layout.set_specific_metabolite('PHAg_c',100)
layout.set_specific_metabolite('amylose_c',100) # It is required for the SINK

print("Loading the comets parameters...")
params_package=None
params_global=None
params =initialize_params(params_package, params_global)
print("Creating the comets object...")
comets = c.comets(layout, params)
dirPlot='promicon_output'
comets.run(delete_files=True)
print(comets.run_output)

In [ ]:
metabolites=['co2_e','o2_e','C80aPHA_e','sucr_e']
df2=make_df_and_graph(["Manoli","Synechococcus"], metabolites, comets, 1000)
df2

# Consorcio R palustris -Synechococcus























In [ ]:

farmer=c.model()
farmer.read_cobra_model("PROMICON/iJN785_sucrose.xml")
farmer.change_bounds('SPP',0.29,1000)
farmer.change_bounds('SUCR',0,0)

producer=c.model()
producer.read_cobra_model("PROMICON/iAN1128_sucrose.xml")

farmer.initial_pop=[0,0,0.2]
producer.initial_pop=[0,0,0.01]
layout=initialize_layout([producer,farmer])
set_layout_media_palustris(layout)
set_layout_media_synechoccus(layout)
set_layout_media_putida(layout)
#layout.set_specific_metabolite("photon_e",1000)
layout.set_specific_metabolite("n2_e",40)
layout.set_specific_metabolite("ac_e",0)
layout.set_specific_metabolite("sucr_e",0)


layout.set_specific_metabolite('amylose_c',100) # It is required for the SINK

print("Loading the comets parameters...")
params_package=None
params_global=None
params =initialize_params(params_package, params_global)
print("Creating the comets object...")
comets = c.comets(layout, params)
dirPlot='promicon_output'
comets.run(delete_files=True)
print(comets.run_output)

In [ ]:
metabolites=['co2_e','h2_e','n2_e','sucr_e']
df2=make_df_and_graph(["R.palustris","Synechococcus"], metabolites, comets, 1000)
df2

In [ ]:
model=cobra.io.read_sbml_model("PROMICON/iAN1128_sucrose.xml")
model.reactions.EX_ac_e.lower_bound=0
model.reactions.XR229.bounds=(0.03,0.03)
model.summary()

# FLYCOP ejecución con el consorcio Syneco-R.palustris

In [3]:
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output
def FLYCOP_space():
    # Build Configuration Space which defines all parameters and their ranges
    cs = ConfigurationSpace()
    x1 = UniformFloatHyperparameter("biomassPalustris", 0.0001,0.1, default_value=0.01)
    x2 = UniformFloatHyperparameter("biomassSynechococcus", 0.0001,0.3, default_value=0.01)
    x3 = UniformFloatHyperparameter("n2",1,40, default_value=20)
    #x4 = CategoricalHyperparameter("GRPalustris", [0.0001,0.001,0.014,0.02,0.03,0.04,0.05,0.1], default_value=0.014)
    #x5 = CategoricalHyperparameter("GRSynechococcus", [0.0001,0.001,0.01,0.02,0.05,0.075,0.1,0.125,0.15,0.175], default_value=0.01)
    cs.add_hyperparameters([x1, x2, x3])
    scenario = Scenario({"deterministic": "true",
                         "use-instances" : "false",
                         "run_obj": "quality",  # we optimize quality (alternatively runtime)
                         "runcount-limit": 100,  # max. number of function evaluations; for this example set to a low number
                         "cs": cs,
                         "shared_model": True,
                         "input_psmac_dirs": "PROMICON/PROMICON_runs_*"
                         })
    return(scenario)

def FLYCOP_schema(parameters):
    #load parameters
    biomass1=parameters["biomassPalustris"]
    biomass2=parameters["biomassSynechococcus"]
    n2=parameters["n2"]
    metabolites=['co2_e','h2_e','n2_e','sucr_e']
    strains=["R.palustris","Synechococcus"]
    # create COMETS run
    farmer=c.model()
    farmer.read_cobra_model("PROMICON/iJN785_sucrose.xml")
    farmer.change_bounds('SPP',0.29,1000)
    farmer.change_bounds('SUCR',0,0)

    producer=c.model()
    producer.read_cobra_model("PROMICON/iAN1128_sucrose.xml")

    farmer.initial_pop=[0,0,biomass2]
    producer.initial_pop=[0,0,biomass1]
    layout=initialize_layout([producer,farmer])
    set_layout_media_palustris(layout)
    set_layout_media_synechoccus(layout)
    set_layout_media_putida(layout)

    layout.set_specific_metabolite("n2_e",n2)
    layout.set_specific_metabolite("ac_e",0)
    layout.set_specific_metabolite("sucr_e",0)


    layout.set_specific_metabolite('amylose_c',100) # It is required for the SINK

    print("Loading the comets parameters...")
    params_package=None
    params_global=None
    params =initialize_params(params_package, params_global)
    print("Creating the comets object...")
    comets = c.comets(layout, params)
    dirPlot='promicon_output'
    comets.run(delete_files=True)
    
    comets.total_biomass.to_csv('Total_biomass_log_template.txt',sep='\t',index=False)
    endCycle=find_end_cycle(comets.total_biomass)
        
    print("The last cycle is: %d"%endCycle)
    maxCycles=endCycle-1
    # Plot
    df2=make_df_and_graph(strains, metabolites, comets, endCycle)
    sucrose=float(df2.at[maxCycles-1,'sucr_e'])
    
    final_biomass1=float(df2.at[maxCycles-1,'R.palustris'])
    final_biomass2=float(df2.at[maxCycles-1,'Synechococcus'])
    
    h2=float(df2.at[maxCycles-1,'h2_e'])
    title='-R.palustris: '+str(biomass1)+'-Synechococcus: '+str(biomass2)+'-Sucr: '+str(sucrose)+'-H2: '+str(h2)
    
    # Fitness
    fitness=h2-sucrose
    print("Configuration\tfitness\th2(mM)\tsucr(mM)\tBiomassFarmer\tBiomassLaborer\tendCycle")
    print(str(biomass1)+','+str(biomass2)+','+str(n2)+'\t'+str(fitness)+'\t'+str(h2)+'\t'+str(sucrose)+'\t'+str(final_biomass1)+'\t'+str(final_biomass2)+'\t'+str(endCycle))
    print("Getting fluxes:End")
    fitness_inverse=-1*fitness
    return fitness_inverse,0

def FLYCOP_run(scenario):
    print("Optimizing! Depending on your machine, this might take a few minutes.")
    smac = SMAC4BO(scenario=scenario,
                   rng=np.random.RandomState(123),
                   tae_runner=FLYCOP_schema,
                   )
    result=smac.optimize()
    return(result)


    
    


In [7]:
parameters={"biomassPalustris":0.04,
    "biomassSynechococcus":0.3,
    "n2":20}
FLYCOP_schema(parameters)

building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 918
Configuration	fitness	h2(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.04,0.3,20	33.258942875276	33.283638935	0.024696059723999998	4.9882283751	0.85253203162	918
Getting fluxes:End


(-33.258942875276, 0)

In [8]:
# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC4BO(scenario=FLYCOP_space(),
                   rng=np.random.RandomState(123),
                   tae_runner=FLYCOP_schema,
                   )
start = time.time()
smac.optimize()
end = time.time()

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2023-04-25_15:36:32_907938
INFO:smac.facade.smac_bo_facade.SMAC4BO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!


Optimizing! Depending on your machine, this might take a few minutes.


INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 24 configurations
INFO:smac.facade.smac_bo_facade.SMAC4BO:<class 'smac.facade.smac_bo_facade.SMAC4BO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 1000
Configuration	fitness	h2(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.050050000000000004,0.15005,20.5	97.202996833788	97.216679005	0.013682171211999999	5.1219840845	0.47234143085	1000
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: -97.2030
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


...

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 38.793995 sec, available: 0.000010 sec)
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 1000
Configuration	fitness	h2(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.09999967390684432,0.23604916102660425,28.66493815263159	198.267505234404	198.28902916	0.021523925596	7.1920326908	0.74305763725	1000
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 39.616261 sec, available: 0.000010 sec)
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 1000
Configuration	fitness	h2(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.09999967390684432,0.2360941729222955,28.66950906589241	198.270766130036	198.29229416	0.021528029964000002	7.1931641337	0.74319932991	1000
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 39.689194 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 12
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 4481.38 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 3972.09 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_bo_facade.SMAC4BO:Final Incumbent: Configuration:
  biomassPalustris, Value: 0.09999967390684432
  biomassSynechococcus, Value: 0.23654467940501417
  n2, Value: 28.670358008058937

INFO:smac.facade.smac_bo_facade.SMAC4BO:E

# FLYCOP ejecución con el consorcio Syneco-P.putida

In [18]:
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output
def FLYCOP_space():
    # Build Configuration Space which defines all parameters and their ranges
    cs = ConfigurationSpace()
    x1 = UniformFloatHyperparameter("biomassPutida", 0.0001,0.1, default_value=0.01)
    x2 = UniformFloatHyperparameter("biomassSynechococcus", 0.0001,0.3, default_value=0.01)
    x3 = UniformFloatHyperparameter("nh4",1,40, default_value=20)
    #x4 = CategoricalHyperparameter("GRPalustris", [0.0001,0.001,0.014,0.02,0.03,0.04,0.05,0.1], default_value=0.014)
    #x5 = CategoricalHyperparameter("GRSynechococcus", [0.0001,0.001,0.01,0.02,0.05,0.075,0.1,0.125,0.15,0.175], default_value=0.01)
    cs.add_hyperparameters([x1, x2, x3])
    scenario = Scenario({"deterministic": "true",
                         "use-instances" : "false",
                         "run_obj": "quality",  # we optimize quality (alternatively runtime)
                         "runcount-limit": 100,  # max. number of function evaluations; for this example set to a low number
                         "cs": cs,
                         "shared_model": True,
                         "input_psmac_dirs": "PROMICON/PROMICON_runs_*"
                         })
    return(scenario)

def FLYCOP_schema(parameters):
    #load parameters
    biomass1=parameters["biomassPutida"]
    biomass2=parameters["biomassSynechococcus"]
    nh4=parameters["nh4"]
    metabolites=['co2_e','C80aPHA_e','nh4_e','sucr_e']
    strains=["P.putida","Synechococcus"]
    # create COMETS run
    
    
    farmer=c.model()
    farmer.read_cobra_model("PROMICON/iJN785_sucrose.xml")
    farmer.change_bounds('SPP',0.25,1000)
    farmer.change_bounds('SUCR',0,0)

    producer=c.model()
    producer.read_cobra_model("PROMICON/manoli_sucrose.xml")
    producer.change_bounds('C80aPHAtex',2,1000)
    producer.change_bounds('EX_C80aPHA_e',2,1000)



    farmer.initial_pop=[0,0,biomass2]
    producer.initial_pop=[0,0,biomass1]
    layout=initialize_layout([producer,farmer])
    layout.set_specific_metabolite('amylose_c',100) # It is required for the SINK
    set_layout_media_synechoccus(layout)
    set_layout_media_putida(layout)

    layout.set_specific_metabolite("nh4_e",nh4)
    layout.set_specific_metabolite("ac_e",0)
    layout.set_specific_metabolite("sucr_e",0)


    layout.set_specific_metabolite('amylose_c',100) # It is required for the SINK

    print("Loading the comets parameters...")
    params_package=None
    params_global=None
    params =initialize_params(params_package, params_global)
    print("Creating the comets object...")
    comets = c.comets(layout, params)
    dirPlot='promicon_output'
    comets.run(delete_files=True)
    
    comets.total_biomass.to_csv('Total_biomass_log_template.txt',sep='\t',index=False)
    endCycle=find_end_cycle(comets.total_biomass)
        
    print("The last cycle is: %d"%endCycle)
    maxCycles=endCycle-1
    # Plot
    df2=make_df_and_graph(strains, metabolites, comets, endCycle)
    sucrose=float(df2.at[maxCycles-1,'sucr_e'])
    
    final_biomass1=float(df2.at[maxCycles-1,'P.putida'])
    final_biomass2=float(df2.at[maxCycles-1,'Synechococcus'])
    
    PHA=float(df2.at[maxCycles-1,'C80aPHA_e'])
    title='-P.putida: '+str(biomass1)+'-Synechococcus: '+str(biomass2)+'-Sucr: '+str(sucrose)+'-PHA: '+str(PHA)
    
    # Fitness
    fitness=PHA-sucrose
    print("Configuration\tfitness\tPHA(mM)\tsucr(mM)\tBiomassFarmer\tBiomassLaborer\tendCycle")
    print(str(biomass1)+','+str(biomass2)+','+str(nh4)+'\t'+str(fitness)+'\t'+str(PHA)+'\t'+str(sucrose)+'\t'+str(final_biomass1)+'\t'+str(final_biomass2)+'\t'+str(endCycle))
    print("Getting fluxes:End")
    fitness_inverse=-1*fitness
    return fitness_inverse,0

def FLYCOP_run(scenario):
    print("Optimizing! Depending on your machine, this might take a few minutes.")
    smac = SMAC4BO(scenario=scenario,
                   rng=np.random.RandomState(123),
                   tae_runner=FLYCOP_schema,
                   )
    result=smac.optimize()
    return(result)



In [14]:
parameters={"biomassPutida":0.04,
    "biomassSynechococcus":0.3,
    "nh4":20}
FLYCOP_schema(parameters)

building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 693
Configuration	fitness	PHA(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.04,0.3,20	14.541126864	14.541126864	0.0	0.20593022514	1.3955331606	693
Getting fluxes:End


(-14.541126864, 0)

In [19]:
# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC4BO(scenario=FLYCOP_space(),
                   rng=np.random.RandomState(123),
                   tae_runner=FLYCOP_schema,
                   )
start = time.time()
smac.optimize()
end = time.time()

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2023-04-26_10:14:17_341356
INFO:smac.facade.smac_bo_facade.SMAC4BO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 24 configurations
INFO:smac.facade.smac_bo_facade.SMAC4BO:<class 'smac.facade.smac_bo_facade.SMAC4BO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


Optimizing! Depending on your machine, this might take a few minutes.
building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 950
Configuration	fitness	PHA(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.050050000000000004,0.15005,20.5	15.706695202999999	15.706695202999999	0.0	0.18642567233	1.2475121113	950
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: -15.7067
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 1000
Configuration	fitness	PHA(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.07502500000000001,0.075075,30.25	14.942284603058	14.959985000000001	0.017700396942	0.075025	0.7096112257	1000
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 138.409105 sec, available: 0.000010 sec)
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


...

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 68.778492 sec, available: 0.000010 sec)
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 1000
Configuration	fitness	PHA(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.09999992372617501,0.0748869802124282,10.221694216456072	19.922328723394	19.939984791	0.017656067605999998	0.099999923726	0.70783405684	1000
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 68.462444 sec, available: 0.000010 sec)
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 1000
Configuration	fitness	PHA(mM)	sucr(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.09999992372617501,0.1334372009985119,17.727067551677095	19.908524365766002	19.939984791	0.031460425234	0.099999923726	1.2612525575	1000
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 69.826042 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 8
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 7243.14 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 6657.67 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_bo_facade.SMAC4BO:Final Incumbent: Configuration:
  biomassPutida, Value: 0.09999992372617501
  biomassSynechococcus, Value: 0.03348631563694331
  nh4, Value: 23.548002533786054

INFO:smac.facade.smac_bo_facade.SMAC4BO:Esti

# FLYCOP ejecución con el consorcio Synechocystis-E.coli

In [2]:
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output
def FLYCOP_space():
    # Build Configuration Space which defines all parameters and their ranges
    cs = ConfigurationSpace()
    x1 = UniformFloatHyperparameter("biomassEcoli", 0.0001,0.1, default_value=0.01)
    x2 = UniformFloatHyperparameter("biomassSynechocystis", 0.0001,0.3, default_value=0.01)
    x3 = UniformFloatHyperparameter("nh4",1,40, default_value=20)
    #x4 = CategoricalHyperparameter("GRPalustris", [0.0001,0.001,0.014,0.02,0.03,0.04,0.05,0.1], default_value=0.014)
    #x5 = CategoricalHyperparameter("GRSynechococcus", [0.0001,0.001,0.01,0.02,0.05,0.075,0.1,0.125,0.15,0.175], default_value=0.01)
    cs.add_hyperparameters([x1, x2, x3])
    scenario = Scenario({"deterministic": "true",
                         "use-instances" : "false",
                         "run_obj": "quality",  # we optimize quality (alternatively runtime)
                         "runcount-limit": 100,  # max. number of function evaluations; for this example set to a low number
                         "cs": cs,
                         "shared_model": True,
                         "input_psmac_dirs": "PROMICON/PROMICON_runs_*"
                         })
    return(scenario)

def FLYCOP_schema(parameters):
    #load parameters
    biomass1=parameters["biomassEcoli"]
    biomass2=parameters["biomassSynechocystis"]
    nh4=parameters["nh4"]
    metabolites=['co2_e','btoh_e','nh4_e','o2_e','sucr_e','ac_e']
    strains=["E.coli","Synechocystis"]
    # create COMETS run
    
    farmer=c.model()
    farmer.read_cobra_model('PROMICON/iJN678_acetate_mutant.xml')
    producer=c.model()
    producer.read_cobra_model('PROMICON/iML1515_butanol.xml')
    producer.change_bounds('BTOHtex',0.25,1000)
    producer.change_bounds('EX_btoh_e',0,0.25)

    farmer.initial_pop=[0,0,biomass2]
    producer.initial_pop=[0,0,biomass1]
    layout=initialize_layout([producer,farmer])
    set_layout_media_ecoli(layout)
    set_layout_media_synechocystis(layout)
    layout.set_specific_metabolite('rdmbzi_c',100)
    layout.set_specific_metabolite("nh4_e",nh4)
    layout.set_specific_metabolite("ac_e",0)
    layout.set_specific_metabolite("sucr_e",0)

    print("Loading the comets parameters...")
    params_package=None
    params_global=None
    params =initialize_params(params_package, params_global)
    print("Creating the comets object...")
    comets = c.comets(layout, params)
    dirPlot='promicon_output'
    comets.run(delete_files=True)
    
    comets.total_biomass.to_csv('Total_biomass_log_template.txt',sep='\t',index=False)
    endCycle=find_end_cycle(comets.total_biomass)
        
    print("The last cycle is: %d"%endCycle)
    maxCycles=endCycle-1
    # Plot
    df2=make_df_and_graph(strains, metabolites, comets, endCycle)
    acetate=float(df2.at[maxCycles-1,'ac_e'])
    
    final_biomass1=float(df2.at[maxCycles-1,'E.coli'])
    final_biomass2=float(df2.at[maxCycles-1,'Synechocystis'])
    
    Butanol=float(df2.at[maxCycles-1,'btoh_e'])
    title='-E.coli: '+str(biomass1)+'-Synechococystis: '+str(biomass2)+'-Acetate: '+str(acetate)+'-Butanol: '+str(Butanol)
    
    # Fitness
    fitness=Butanol-acetate
    print("Configuration\tfitness\tButanol(mM)\tacetate(mM)\tBiomassFarmer\tBiomassLaborer\tendCycle")
    print(str(biomass1)+','+str(biomass2)+','+str(nh4)+'\t'+str(fitness)+'\t'+str(Butanol)+'\t'+str(acetate)+'\t'+str(final_biomass1)+'\t'+str(final_biomass2)+'\t'+str(endCycle))
    print("Getting fluxes:End")
    fitness_inverse=-1*fitness
    return fitness_inverse,0

In [6]:
parameters={"biomassEcoli":0.04,
    "biomassSynechocystis":0.3,
    "nh4":20}
FLYCOP_schema(parameters)

building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 166
Configuration	fitness	Butanol(mM)	acetate(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.04,0.3,20	0.157	0.157	0.0	0.04	1.1787126658	166
Getting fluxes:End


(-0.157, 0)

In [7]:
# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC4BO(scenario=FLYCOP_space(),
                   rng=np.random.RandomState(123),
                   tae_runner=FLYCOP_schema,
                   )
start = time.time()
smac.optimize()
end = time.time()

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2023-05-16_10:41:16_258625
INFO:smac.facade.smac_bo_facade.SMAC4BO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 24 configurations
INFO:smac.facade.smac_bo_facade.SMAC4BO:<class 'smac.facade.smac_bo_facade.SMAC4BO'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


Optimizing! Depending on your machine, this might take a few minutes.
building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 230
Configuration	fitness	Butanol(mM)	acetate(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.050050000000000004,0.15005,20.5	0.27652625	0.27652625	0.0	0.05005	1.0293112329	230
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: -0.2765
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 301
Configuration	fitness	Butanol(mM)	acetate(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.07502500000000001,0.075075,30.25	0.478284375	0.478284375	0.0	0.075025	0.95221518365	301
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:Challenger (-0.4783) is better than incumbent (-0.2765) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  biomassEcoli : 0.050050000000000004 -> 0.07502500000000001
INFO:smac.intensification.intensification.Intensifier:  biomassSynechocystis : 0.15005 -> 0.075075
INFO:smac.intensification.intensification.Intensifier:  nh4 : 20.5 -> 30.25
INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 124.323195 sec, available: 0.000010 sec)
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


...

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 68.555669 sec, available: 0.000010 sec)
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 1000
Configuration	fitness	Butanol(mM)	acetate(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.09999969255390692,0.0001122333530569219,1.012364480190081	0.6663621850834001	0.6674979478	0.0011357627165999998	0.099999692554	0.67269311619	1000
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 71.293796 sec, available: 0.000010 sec)
INFO:smac.optimizer.smbo.SMBO:Shared model mode: Finished loading new runs, found 0 new runs.


building empty layout model
models will need to be added with layout.add_model()
Loading the comets parameters...
Creating the comets object...

Running COMETS simulation ...
Done!
The last cycle is: 1000
Configuration	fitness	Butanol(mM)	acetate(mM)	BiomassFarmer	BiomassLaborer	endCycle
0.09999910522286458,0.00010149335035511456,16.716811312583804	0.6364672181962999	0.6374942957999999	0.0010270776037	0.099999105223	0.60832075549	1000
Getting fluxes:End


INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 68.049291 sec, available: 0.000010 sec)
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 3
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 100 / 100.0
INFO:smac.stats.stats.Stats:#Configurations: 100
INFO:smac.stats.stats.Stats:Used wallclock time: 7248.24 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 6649.37 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_bo_facade.SMAC4BO:Final Incumbent: Configuration:
  biomassEcoli, Value: 0.09999800662908127
  biomassSynechocystis, Value: 0.00021944674616659426
  nh4, Value: 21.029289136616452

INFO:smac.facade.smac_bo_facade.SMAC4BO:Es